<a href="https://colab.research.google.com/github/fsndzomga/Deep-Learning-With-Python/blob/main/Predict_the_next_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import string
import numpy as np
import re
import random
from sklearn.linear_model import SGDRegressor
import pickle
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import json
import tensorflow as tf

In [2]:
def preprocessing_text(text_file_path):
    # Loading my text file
    text = open(text_file_path, "r").read().lower()

    # Remove numbers using isdigit()
    text = ''.join([c for c in text if not c.isdigit()])

    # Remove punctuation using str.translate() and string.punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Replace multiple spaces with single space
    text = re.sub(' +', ' ', text)

    # Replace line breaks with single space
    text = text.replace("\n", " ")

    return text

In [3]:
text = preprocessing_text("/content/drive/MyDrive/Machine Learning Files/bible.txt")

In [4]:
def encoding(text):
    # Generate the list of unique words in my text
    unique_words = set(text.split(" "))

    # Get the number of elements in the set
    num_elements = len(unique_words)

    with open("/content/drive/MyDrive/Machine Learning Files/encoding.json", "r") as file:
      json_data = file.read()
      text_encoding = json.loads(json_data)


    # use the text encoding to map the original corpus
    encoded_text = [text_encoding[word] for word in text.split(" ")]

    return encoded_text

In [5]:
encoded_text = encoding(text)

In [6]:
def add_zeros_to_array(array, m):
    # Calculate the number of zeros to add
    num_zeros = m - len(array)

    # Add zeros to the beginning of the array
    result = [0] * num_zeros + array

    return result

def features_targets(encoded_text, num_words):

    features = []
    targets = []

    for i in range(len(encoded_text) - num_words + 1):
        subtext = encoded_text[i:i + num_words + 1]
        for j in range(1, len(subtext)):
            features.append(add_zeros_to_array(subtext[0:j], num_words))
            targets.append(subtext[j])

    return [features, targets]

In [7]:
features, targets = features_targets(encoded_text, 5)

In [8]:
# Transforming features and targets into numpy arrays

features = np.array(features)

targets = np.array(targets)

In [9]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the input data
scaler.fit(features)

# Apply the transformation to the input data
features_scaled = scaler.transform(features)

In [10]:
# # Create a OneHotEncoder object
# encoder = OneHotEncoder()

# # Fit the encoder to the input data
# encoder.fit(targets.reshape(-1, 1))

# # Apply the transformation to the input data
# targets = encoder.transform(targets.reshape(-1, 1)).toarray()

In [11]:
features_scaled.shape

(4202939, 5)

In [34]:
from keras.models import Sequential
from keras.layers import Dense
# Training a scikit learn regression model

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, targets, test_size=0.2, random_state=42)

# Create a sequential model
model = Sequential()

# Add the first dense layer with 128 neurons and input shape (input_shape)
model.add(Dense(40000, input_shape=(5,), activation='relu'))

# Add 7 more dense layers with 128 neurons each
for i in range(10):
    model.add(Dense(200, activation='relu'))

# # Add the output layer with a single neuron and sigmoid activation
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
# Add the output layer with a single neuron and linear activation
model.add(Dense(1, activation='linear'))

# Compile the model with mean squared error as the loss function
model.compile(loss='mean_squared_error', optimizer='adam')

In [35]:
model.fit(X_train, y_train, epochs=5, batch_size=10000)

Epoch 1/5


ResourceExhaustedError: ignored